In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:

import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_review = read_file(spark, "parquet", f'{bronze}/order_reviews')

In [0]:
df_nullfill = df_review.withColumn("review_comment_message", coalesce(col("review_comment_message"), lit('no review')))

df_cast = df_nullfill.filter("review_score RLIKE '^[0-9]+$'")
columns = {"review_score": "integer"}
df_cast_col = f_cast_col(df_cast, columns)

df_dedup = deduplicate(df_cast_col, ["review_id"])
df_dropna = drop_required_nulls(df_dedup, ["order_id"])
df_filter = df_dropna.filter("review_score between 1 and 5") \
    .withColumn("negative_review_flag", when(col("review_score") <= 2, 1).otherwise(0))
# display(df_filter)

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "review_orders"}}

df_final_rorders = enrich_add_col(df_filter, columns)
# df_cast_col.display()

In [0]:
# Write data delta file /delta table in silver location

write_file(df_final_rorders,f"{silver}/silver_reviews")

write_delta_liquid(df_final_rorders, f"{catalog}.{schema}.silver_reviews",["review_id","order_id"])